In [1]:
###################################################
# PROJECT: Rating Product & Sorting Reviews in Amazon
###################################################


In [ ]:
###################################################
# BUSINESS PROBLEM
###################################################

#One of the most important problems in e-commerce is the correct calculation of the points given to the products after sales.
#Another problem is the correct ordering of the comments given to the products.
# THIS PROJECT IS TO SOLVE THESE 2 RATING AND SORTING PROBLEMS ABOVE

In [ ]:
###################################################
# DATASET
###################################################

# This Amazon dataset contains product infos, categories and some other metadata regarding the product.
# Dataset contains  most reviewed products in the electronics category having most user ratings and reviews

# Variables below:
# reviewerID: User ID
# asin: Product ID
# reviewerName: Username
# helpful: Level of helpful review
# reviewText: Review itself
# overall: Product rating
# summary: Review Summary
# unixReviewTime: Review Time
# reviewTime: Raw Review Time
# day_diff: Number of days since assessment
# helpful_yes: The number of times the review was found useful
# total_vote: Number of votes given to the review


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
df = pd.read_csv("amazon_review.csv")
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00000,Four Stars,1406073600,2014-07-23,138,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00000,nothing to really say....,1356220800,2012-12-23,715,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00000,best deal around,1373673600,2013-07-13,513,0,0


In [ ]:
###################################################
# TASK 1: Calculate Average Rating Based on Current Comments and Compare with Existing Average Rating.
###################################################

In [4]:
df["overall"].mean()

4.587589013224822

In [9]:
df['reviewTime'] = pd.to_datetime (df['reviewTime'], dayfirst=True)

In [10]:
current_date = pd.to_datetime (str (df['reviewTime'].max ()))

In [11]:
df["day_diff"] = (current_date - df['reviewTime']).dt.days

In [12]:
# calculation of time-based weighted average
def time_based_weighted_average(dataframe, w1=50, w2=25, w3=15, w4=10):
    return dataframe.loc[dataframe["day_diff"] <= dataframe["day_diff"].quantile (0.25), "overall"].mean () * w1 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile (0.25)) & (
            dataframe["day_diff"] <= dataframe["day_diff"].quantile (0.50)), "overall"].mean () * w2 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile (0.50)) & (
            dataframe["day_diff"] <= dataframe["day_diff"].quantile (0.75)), "overall"].mean () * w3 / 100 + \
           dataframe.loc[(dataframe["day_diff"] > dataframe["day_diff"].quantile (0.75)), "overall"].mean () * w4 / 100

In [13]:
time_based_weighted_average(df)

4.637306192407316

In [14]:
# Another time based weighted average
time_based_weighted_average(df, w1=28, w2=26, w3=24, w4=22)

4.595593165128118

In [ ]:
#TASK 2: Specify top 20 Reviews for the Product which can be Displayed on the Product Detail Page

In [15]:
# Here we create a new variable as "helpful_no" , in order to see unhelpfully reviews
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]]
# Now we cam see the updated dataframe
df.head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime
0,NaN,4.00000,Four Stars,0,0,0,2014-07-23
1,0mie,5.00000,MOAR SPACE!!!,0,0,0,2013-10-25
2,1K3,4.00000,nothing to really say....,0,0,0,2012-12-23
3,1m2,5.00000,Great buy at this price!!! *** UPDATE,0,0,0,2013-11-21
4,2&amp;1/2Men,5.00000,best deal around,0,0,0,2013-07-13


In [16]:
# We should calculate :  score_pos_neg_diff, score_average_rating and wilson_lower_bound scores

#wilson_lower_bound function
def wilson_lower_bound(up, down, confidence=0.95):
    """
    Wilson Lower Bound Score hesapla

    - Bernoulli parametresi p için hesaplanacak güven aralığının alt sınırı WLB skoru olarak kabul edilir.
    - Hesaplanacak skor ürün sıralaması için kullanılır.
    - Not:
    Eğer skorlar 1-5 arasıdaysa 1-3 negatif, 4-5 pozitif olarak işaretlenir ve bernoulli'ye uygun hale getirilebilir.
    Bu beraberinde bazı problemleri de getirir. Bu sebeple bayesian average rating yapmak gerekir.

    Parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [17]:
def score_up_down_diff(up, down):
    return up - down


def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [19]:
#####################
# score_pos_neg_diff
#####################


df["score_pos_neg_diff"] = df.apply(lambda x: score_up_down_diff(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_pos_neg_diff", ascending=False).head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85


In [20]:
# score_average_rating
df["score_average_rating"] = df.apply(lambda x: score_average_rating(x["helpful_yes"], x["helpful_no"]), axis=1)
df.sort_values("score_average_rating", ascending=False).head()

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000


In [21]:
# wilson_lower_bound
df["wilson_lower_bound"] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"], x["helpful_no"]), axis=1)

# Let's see the top 20
df.sort_values("wilson_lower_bound", ascending=False).head(20)

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
...,...,...,...,...,...,...,...,...,...,...
1072,Crysis Complex,5.00000,Works wonders for the Galaxy Note 2!,5,0,5,2012-05-10,5,1.00000,0.56552
2583,J. Wong,5.00000,Works Great with a GoPro 3 Black!,5,0,5,2013-08-06,5,1.00000,0.56552
121,A. Lee,5.00000,ready for use on the Galaxy S3,5,0,5,2012-05-09,5,1.00000,0.56552
1142,Daniel Pham(Danpham_X @ yahoo. com),5.00000,Great large capacity card,5,0,5,2014-02-04,5,1.00000,0.56552
